In [2]:
import os
import pdfplumber


<span style="color: red; font-size: 2em;">Here we extract the words and the (x,y) data from a pdf page</span>


In [9]:
#enter path to cycle 74 pdf here
file_path=r"C:\\Users\\apagano\\OneDrive - JBS International\\Desktop\\NAWS PDFs\\ENG2012CY74.pdf"
with pdfplumber.open(file_path) as pdf:
    first_page = pdf.pages[0]
    second_page = pdf.pages[1]
    page = first_page.extract_words()
    
for word in page:
    print(word)
        

{'text': 'S:\\NAWSDOC\\CYCLE74\\ENGLISHCY74\\ENG2012CY74.wpd', 'x0': 28.79998848, 'x1': 194.60940167573318, 'top': 15.402310639199982, 'doctop': 15.402310639199982, 'bottom': 21.402308239199897, 'upright': True, 'height': 5.999997599999915, 'width': 165.80941319573319, 'direction': 'ltr'}
{'text': 'ENGLISH', 'x0': 28.79998848, 'x1': 73.05722528765098, 'top': 23.182027527312016, 'doctop': 23.182027527312016, 'bottom': 33.14202354331201, 'upright': True, 'height': 9.959996015999991, 'width': 44.25723680765098, 'direction': 'ltr'}
{'text': 'Cycle', 'x0': 28.79998848, 'x1': 55.20706403651138, 'top': 35.062022775312016, 'doctop': 35.062022775312016, 'bottom': 45.02201879131201, 'upright': True, 'height': 9.959996015999991, 'width': 26.407075556511383, 'direction': 'ltr'}
{'text': '74,', 'x0': 57.959980755118124, 'x1': 71.76351819105278, 'top': 35.062022775312016, 'doctop': 35.062022775312016, 'bottom': 45.02201879131201, 'upright': True, 'height': 9.959996015999991, 'width': 13.803537435934

<span style="color: red; font-size: 2em;">Here we combine lines based off stringing together words that are close together and horizonal to eachother</span>

In [14]:
sentences = []  # To store the sentences
lamdaXs = 30     # Horizontal spacing threshold
lamdaYs = 5      # Vertical alignment threshold

word_index = 0
while word_index < len(page) - 1:
    sentence = [page[word_index]]  # Start with the current word
    ahead_index = word_index + 1

    while ahead_index < len(page):
        current_word = sentence[-1]
        next_word = page[ahead_index]

        # Check horizontal and vertical proximity
        close_x = current_word['x1'] < next_word['x0'] < current_word['x1'] + lamdaXs
        aligned_y = abs(current_word['top'] - next_word['top']) <= lamdaYs

        if close_x and aligned_y:
            sentence.append(next_word)
            ahead_index += 1
        else:
            break

    sentences.append(sentence)
    word_index = ahead_index  # Jump to the next unprocessed word
for s in sentences:
    print("\n" + " ".join([word['text'] for word in s]))
    print(s)


S:\NAWSDOC\CYCLE74\ENGLISHCY74\ENG2012CY74.wpd
[{'text': 'S:\\NAWSDOC\\CYCLE74\\ENGLISHCY74\\ENG2012CY74.wpd', 'x0': 28.79998848, 'x1': 194.60940167573318, 'top': 15.402310639199982, 'doctop': 15.402310639199982, 'bottom': 21.402308239199897, 'upright': True, 'height': 5.999997599999915, 'width': 165.80941319573319, 'direction': 'ltr'}]

ENGLISH
[{'text': 'ENGLISH', 'x0': 28.79998848, 'x1': 73.05722528765098, 'top': 23.182027527312016, 'doctop': 23.182027527312016, 'bottom': 33.14202354331201, 'upright': True, 'height': 9.959996015999991, 'width': 44.25723680765098, 'direction': 'ltr'}]

Cycle 74, FALL 2012
[{'text': 'Cycle', 'x0': 28.79998848, 'x1': 55.20706403651138, 'top': 35.062022775312016, 'doctop': 35.062022775312016, 'bottom': 45.02201879131201, 'upright': True, 'height': 9.959996015999991, 'width': 26.407075556511383, 'direction': 'ltr'}, {'text': '74,', 'x0': 57.959980755118124, 'x1': 71.76351819105278, 'top': 35.062022775312016, 'doctop': 35.062022775312016, 'bottom': 45.02

<span style="color: red; font-size: 2em;">Here we take our lines and stack ones that are either centrally aligned, or aligned along the left side, and that are directly below eachother</span>

In [19]:
multiLineSentences = []
lamdaXc = 40  # Horizontal center alignment threshold
lamdaYc = 20  # Vertical distance threshold

used = set()

for sentence_i in range(len(sentences)):
    if sentence_i in used:
        continue

    base = sentences[sentence_i].copy()

    for ahead_i in range(sentence_i + 1, len(sentences)):
        if ahead_i in used:
            continue

        candidate = sentences[ahead_i]

        # Horizontal center positions
        xcent_base = (base[0]['x0'] + base[-1]['x1']) / 2
        xcent_cand = (candidate[0]['x0'] + candidate[-1]['x1']) / 2

        # Check alignment
        vertically_close = (base[-1]['bottom'] < candidate[-1]['top'] < base[-1]['bottom'] + lamdaYc)
        horizontally_centered = abs(xcent_base - xcent_cand) <= lamdaXc
        horizontally_left_aligned = abs(base[0]['x0'] - candidate[0]['x0']) <= lamdaXc

        if vertically_close and (horizontally_centered or horizontally_left_aligned):
            base.extend(candidate)
            used.add(ahead_i)

    multiLineSentences.append(base)


for sent in multiLineSentences:
    print(" ".join([word['text'] for word in sent]))
    print(f'\n\n\n\n\n')


S:\NAWSDOC\CYCLE74\ENGLISHCY74\ENG2012CY74.wpd ENGLISH Cycle 74, FALL 2012 OMB NO. 1205-0453 EXPIRATION DATE: 10/31/2013 [REV. Sep 11, 2012]






7 4 FARM WORKER ID






COUNTY






[FOR OFFICE USE ONLY]






NATIONAL AGRICULTURAL WORKERS SURVEY - 2012 (“NAWS”)






CS2 DATE:






/






/






[FOR OFFICE USE ONLY] CROP CODE






CS5 CROP:






CS6 TASK: LANGUAGE DURING INTERVIEW: __________________






TASK CODE






GN:






ID:






IF GN REFERRED TO CONTRACTOR, GROWER OR OTHER, WRITE INFORMATION) NAME : ADDRESS: TELEPHONE:






GN REFERRED TO: 9999 “CONTRACTOR”?: 9999 OTHER GROWER? 9999 OTHER?:_______






_______________________________________ _______________________________________ (_________)___________-________________






WORKER IS ACTUALLY EMPLOYED BY?: 9999 1 GROWER TYPE OF WORK?: 9999






9999 2 CONTRACTOR






1 FIELD WORK 9999






2 NURSERY 9999






3 PACKING HOUSE 9999






7 OTHER:________






FARM WORKER’S NAME:






LOCAL ADDRESS: TELEP